In [1]:
from eda_imports import *
from adjustText import adjust_text

In [2]:
from kleat.post import cluster_clv_parallel

In [3]:
sys.path.insert(0, '../benchmark_scripts/')

from train_arbor import load_df, load_polya_seq_df, map_to_ref, compare
from ml_utils import KARBOR_FEATURE_COLS, calc_precision_recall_curve

In [4]:
clf_pkl_files = sorted(glob.glob(
    '../benchmark_transcriptome/tcga-run-4/benchmark-dt.DecisionTreeClassifier.tree_max_depth_*.pkl'
))

In [5]:
len(clf_pkl_files)

19

In [6]:
clfs_dd = {}
for pkl_file in clf_pkl_files:
    tmax_depth = re.search('tree_max_depth_([0-9]+)', pkl_file).group(1)
    tmax_depth = int(tmax_depth)
    with open(pkl_file, 'rb') as inf:
        print(f'working on {os.path.basename(pkl_file)}', tmax_depth)
        clfs_dd[tmax_depth] = pickle.load(inf)

working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_1.pkl 1
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_10.pkl 10
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_11.pkl 11
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_12.pkl 12
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_13.pkl 13
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_14.pkl 14
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_15.pkl 15
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_16.pkl 16
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_17.pkl 17
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_18.pkl 18
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_19.pkl 19
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_2.pkl 2
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_3.pkl 3
working on benchmark-dt.DecisionTreeClassifier.tree_max_depth_4.pkl 4


In [12]:
%%time
dfs = []
thresholds = np.arange(0, 1.01, 0.05)
for sample_id in ['HBRC4', 'HBRC6', 'UHRC1', 'UHRC2']:
    adf = pd.read_pickle(f'../benchmark_transcriptome/{sample_id}.ml_ready.pkl')
    assert adf[['seqname', 'strand', 'clv']].drop_duplicates().shape[0] == adf.shape[0]

    for tmax_depth, clf in clfs_dd.items():
        adf['pred_prob'] = clf.predict_proba(adf[KARBOR_FEATURE_COLS])[:,1]
        bdf = adf[['seqname', 'strand', 'clv', 'pred_prob']].copy()

        map_cutoff = 50
        df_ref = load_polya_seq_df(sample_id)

        df_res = calc_precision_recall_curve(
            bdf, df_ref,  map_cutoff=50, cluster_cutoff=20, thresholds=thresholds, num_cpus=30)
        df_res['tree_max_depth'] = tmax_depth
        df_res['sample_id'] = sample_id
        dfs.append(df_res)

In [9]:
ndf = pd.concat(dfs)

In [10]:
ndf.to_csv('../benchmark_transcriptome/tcga-run-4/benchmark-dt.pr_curve.csv', index=False)

In [11]:
ndf.shape

(1596, 6)

In [20]:
ndf.sample_id.value_counts()

HBRC6    22
UHRC2    22
UHRC1    22
HBRC4    22
Name: sample_id, dtype: int64

In [17]:
ndf.head()

,recall,prec,f1,threshold,tree_max_depth,sample_id
0,0.337573,0.034208,0.062120,0.00,1,HBRC4
1,0.254211,0.273257,0.263390,0.05,1,HBRC4
2,0.232344,0.367555,0.284712,0.10,1,HBRC4
3,0.188146,0.689252,0.295601,0.15,1,HBRC4
4,0.187552,0.695421,0.295429,0.20,1,HBRC4
